In [1]:
cd ../

/wk171/ytkuo/TCRI_paper


In [2]:
%env CUDA_VISIBLE_DEVICES=-1

env: CUDA_VISIBLE_DEVICES=-1


In [3]:
import tensorflow as tf
from modules.data_handler import ascii_array_to_string
import pandas as pd
import numpy as np

In [4]:
# fet experiment settings from yaml file
from modules.experiment_helper import parse_experiment_settings

experiment_path = 'experiments/ens01.yml'
experiment_settings = parse_experiment_settings(experiment_path)

In [5]:
# load trained model
from modules.model_constructor import create_model_by_experiment_path

model = create_model_by_experiment_path(experiment_path, version='best-MSE')

saved_models/ens01/best-MSE


In [6]:
# load dataset
from modules.data_handler import get_tensorflow_datasets

datasets = get_tensorflow_datasets(**experiment_settings['data'])

Instructions for updating:
The `validate_indices` argument has no effect. Indices are always validated on CPU and never validated on GPU.


In [7]:
df = pd.DataFrame(columns=['now','24h_Vmax', 'pred_24h_Vmax', 'starting_frame'])
for image_sequences, labels, feature,frame_ID_ascii, dV in datasets['test']:
    starting_frame = list(map(ascii_array_to_string, frame_ID_ascii.numpy()))
    pred = np.squeeze(model(image_sequences, feature).numpy())
    V24 = labels.numpy()
    now = V24 - dV.numpy()
    batch_size = pred.shape[0]
    for i in range(batch_size):
        df.loc[df.shape[0]] = [now[i], V24[i], pred[i], starting_frame[i]]   

In [8]:
df = df.sort_values(['starting_frame'])
df['pred_24h_Vmax'] = df['pred_24h_Vmax'].map('{:,.2f}'.format)

In [9]:
df.head(10)

now  24h_Vmax pred_24h_Vmax      starting_frame
652   25.0      40.0         61.18  201701I_2017041412
705   25.0      43.0         58.48  201701I_2017041415
1079  25.0      45.0         51.35  201701I_2017041418
246   28.0      48.0         53.73  201701I_2017041421
2001  30.0      50.0         53.37  201701I_2017041500
1842  33.0      50.0         48.53  201701I_2017041503
903   35.0      50.0         49.90  201701I_2017041506
821   38.0      48.0         52.11  201701I_2017041509
856   40.0      45.0         47.34  201701I_2017041512
314   43.0      40.0         45.22  201701I_2017041515

In [10]:
df.to_csv(f'./saved_csv/{experiment_path[-9:-4]}.csv', index = False)